# PokéGAN Training Notebook
**CSC 487 Final Project Stage 3**

## Install Dependencies


In [4]:
%pip install torch torchvision torchmetrics[image] pyyaml matplotlib tensorboard torch-fidelity kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 19.9 MB/s eta 0:00:00


## Clone Repository

In [5]:
import os

repo_name = 'CSC487-Project'
repo_url = 'https://github.com/BraedenAlonge/CSC487-Project.git'

# Clone or pull repository
if not os.path.exists(repo_name):
    print("Cloning repository...")
    !git clone {repo_url}
else:
    print("Repository already exists. Updating...")
    %cd {repo_name}
    !git pull
    %cd ..

# Move into project directory
if repo_name in os.listdir('.'):
    %cd {repo_name}

print(f"Current directory: {os.getcwd()}")

Cloning repository...
Cloning into 'CSC487-Project'...
remote: Enumerating objects: 172, done.
remote: Counting objects: 100% (172/172), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 172 (delta 87), reused 141 (delta 57), pack-reused 0 (from 0)
Receiving objects: 100% (172/172), 4.88 MiB | 19.06 MiB/s, done.
Resolving deltas: 100% (87/87), done.
/content/CSC487-Project
Current directory: /content/CSC487-Project


## Verify GPU for Training


In [6]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

CUDA available: True
GPU: NVIDIA A100-SXM4-40GB
GPU Memory: 42.47 GB


## Download Dataset (Kaggle Method)

In [7]:
import os
import shutil
import random
from google.colab import files

print("Please upload your kaggle.json file (from Kaggle Account -> API):")
uploaded = files.upload()

# Setup Kaggle Auth
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/ 2>/dev/null
!chmod 600 ~/.kaggle/kaggle.json

print("Downloading from Kaggle...")
!kaggle datasets download -d noodulz/pokemon-dataset-1000 --force

# Clean previous data
if os.path.exists('data/pokemon-dataset-1000'):
    shutil.rmtree('data/pokemon-dataset-1000')
!mkdir -p data

print("Extracting dataset...")
if os.path.exists('pokemon-dataset-1000.zip'):
    # Unzip to a temporary location first to inspect structure
    temp_extract_dir = 'data/temp_extract'
    if os.path.exists(temp_extract_dir): shutil.rmtree(temp_extract_dir)
    !unzip -q pokemon-dataset-1000.zip -d {temp_extract_dir}

    print("Organizing dataset...")
    # Target directories
    base_data_dir = 'data/pokemon-dataset-1000'
    train_dir = os.path.join(base_data_dir, 'train')
    val_dir = os.path.join(base_data_dir, 'val')
    test_dir = os.path.join(base_data_dir, 'test')

    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Find ALL images recursively
    all_images = []
    for root, dirs, files in os.walk(temp_extract_dir):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                all_images.append(os.path.join(root, file))

    print(f"Found {len(all_images)} images total.")

    # Shuffle and Split
    random.shuffle(all_images)
    train_split = int(0.9 * len(all_images))
    val_split = int(0.05 * len(all_images))

    train_imgs = all_images[:train_split]
    val_imgs = all_images[train_split:train_split+val_split]
    test_imgs = all_images[train_split+val_split:]

    print("Moving files to train/val/test folders...")
    # Helper to move files
    def move_files(file_list, target_folder):
        for src in file_list:
            dst = os.path.join(target_folder, os.path.basename(src))
            # Handle duplicate filenames if flattened
            if os.path.exists(dst):
                base, ext = os.path.splitext(os.path.basename(src))
                dst = os.path.join(target_folder, f"{base}_{random.randint(0,9999)}{ext}")
            shutil.move(src, dst)

    move_files(train_imgs, train_dir)
    move_files(val_imgs, val_dir)
    move_files(test_imgs, test_dir)

    # Cleanup temp
    shutil.rmtree(temp_extract_dir)

    print(f"✓ Dataset prepared!")
    print(f"  Train: {len(os.listdir(train_dir))}")
    print(f"  Val: {len(os.listdir(val_dir))}")
    print(f"  Test: {len(os.listdir(test_dir))}")
else:
    print("Error: pokemon-dataset-1000.zip not found! Upload failed...")

Please upload your kaggle.json file (from Kaggle Account -> API):


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/noodulz/pokemon-dataset-1000
License(s): CC0-1.0
 84% 661M/785M [00:00<00:00, 1.73GB/s]
100% 785M/785M [00:00<00:00, 1.73GB/s]
Extracting dataset...
Organizing dataset...
Found 53078 images total.
Moving files to train/val/test folders...
✓ Dataset prepared!
  Train: 47770
  Val: 2653
  Test: 2655


## Training the Model


In [8]:
!python train.py --config configs/final.yaml

2025-12-10 00:48:34.228268: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-10 00:48:34.245363: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765327714.267465    4309 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765327714.274121    4309 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765327714.291156    4309 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

## Viewing Training Progress with Tensorboard (Optional)


In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

## Evaluating the Model


In [11]:
!python eval.py --checkpoint checkpoints/epoch_99_model_best.pt --config configs/final.yaml

Set random seed to 42.
Using device: cuda
Loading checkpoint from checkpoints/epoch_99_model_best.pt
Loaded model from epoch 99
Found 2655 images in data/pokemon-dataset-1000/test
Test dataset size: 2655
Using 1000 real images for evaluation
Generating 1000 samples...
Saving sample images...
Saved image grid to eval_outputs/real_eval_samples.png.
Saved image grid to eval_outputs/fake_eval_samples.png.
Calculating metrics...
/usr/local/lib/python3.12/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: Metric `InceptionScore` will save all extracted features in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
Metrics:
  FID: 78.9190 (lower is better)
  Inception Score: 2.9500 ± 0.1216 (higher is better)
  Diversity Score: 59.8087 (higher is better)
Evaluating discriminator...
Discriminator Stats:
  Real images - Mean score: 0.6446, Std: 0.1046
  Fake images - Mean score: 0.4750, Std: 0.0767
Plotting confusion matrix...
Saved

## Download the Collected Training Data

In [13]:
from google.colab import files
import shutil
shutil.make_archive("outputs", 'zip', "outputs")
files.download("outputs.zip")
shutil.make_archive("eval_outputs", 'zip', "eval_outputs")
files.download("eval_outputs.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Download the Best Model Checkpoint

In [15]:
files.download("checkpoints/epoch_99_model_best.pt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## (Optional) Clear all the data to restart training

In [3]:
!rm -rf /content/CSC487-Project/